In [1]:
import tensorflow as tf 
from tensorflow.python.compiler.tensorrt import trt_convert as trt
print("\033[92mstart load model\033[0m")
converter = trt.TrtGraphConverterV2(input_saved_model_dir="./model/saved_model")
print("\033[92mload model successfully\033[0m")
converter.convert()
converter.save("./model/saved_model_trt")
tf.config.list_physical_devices()

start load model
INFO:tensorflow:Linked TensorRT version: (8, 5, 2)
INFO:tensorflow:Loaded TensorRT version: (8, 5, 2)
load model successfully


2023-08-05 21:20:40.292031: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:984] could not open file to read NUMA node: /sys/bus/pci/devices/0000:00:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-08-05 21:20:40.368412: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:984] could not open file to read NUMA node: /sys/bus/pci/devices/0000:00:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-08-05 21:20:40.368782: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:984] could not open file to read NUMA node: /sys/bus/pci/devices/0000:00:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-08-05 21:20:40.373915: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:984] could not open file to read NUMA node: /sys/bus/pci/devices/0000:00:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-08-05 21:20:40.374307: I tensorflow/compile

2023-08-05 21:20:46.730202: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:984] could not open file to read NUMA node: /sys/bus/pci/devices/0000:00:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-08-05 21:20:46.730456: I tensorflow/core/grappler/devices.cc:66] Number of eligible GPUs (core count >= 8, compute capability >= 0.0): 0
2023-08-05 21:20:46.731005: I tensorflow/core/grappler/clusters/single_machine.cc:358] Starting new session
2023-08-05 21:20:46.732433: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:984] could not open file to read NUMA node: /sys/bus/pci/devices/0000:00:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-08-05 21:20:46.732784: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:984] could not open file to read NUMA node: /sys/bus/pci/devices/0000:00:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-08-05 21:20:46.733039: I tensor

INFO:tensorflow:Clearing prior device assignments in loaded saved model
INFO:tensorflow:Automatic mixed precision has been deactivated.


2023-08-05 21:20:51.975695: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:984] could not open file to read NUMA node: /sys/bus/pci/devices/0000:00:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-08-05 21:20:51.976008: I tensorflow/core/grappler/devices.cc:66] Number of eligible GPUs (core count >= 8, compute capability >= 0.0): 0
2023-08-05 21:20:51.976298: I tensorflow/core/grappler/clusters/single_machine.cc:358] Starting new session
2023-08-05 21:20:51.977399: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:984] could not open file to read NUMA node: /sys/bus/pci/devices/0000:00:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-08-05 21:20:51.977780: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:984] could not open file to read NUMA node: /sys/bus/pci/devices/0000:00:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-08-05 21:20:51.978080: I tensor

INFO:tensorflow:Could not find TRTEngineOp_000_000 in TF-TRT cache. This can happen if build() is not called, which means TensorRT engines will be built and cached at runtime.
INFO:tensorflow:Could not find TRTEngineOp_000_001 in TF-TRT cache. This can happen if build() is not called, which means TensorRT engines will be built and cached at runtime.
INFO:tensorflow:Assets written to: ./model/saved_model_trt/assets


[PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU'),
 PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [ ]:
import tensorflow as tf
from tensorflow.python.compiler.tensorrt import trt_convert as trt
from glob import glob
import numpy as np
import os

# 定义一个函数来读取图像，这个函数需要根据你的实际情况来实现
def load_images(file_name):
    # 这里需要你自己实现如何从file_name读取图像并返回numpy数组
    pass

# 1. 加载已经训练好的TensorFlow模型
saved_model_dir = './model/saved_model' # 请替换为你的模型路径
model = tf.saved_model.load(saved_model_dir)

# 2. 使用TensorRT进行模型优化
params = trt.DEFAULT_TRT_CONVERSION_PARAMS
params = params._replace(max_workspace_size_bytes=(1<<32)) # 修改workspace大小
params = params._replace(precision_mode="FP16") # 设置推理精度
params = params._replace(maximum_cached_engines=100) # 设置最大缓存引擎数量

converter = trt.TrtGraphConverterV2(
    input_saved_model_dir=saved_model_dir,
    conversion_params=params
)
converter.convert()

# 3. 对模型进行序列化并保存
saved_model_dir_trt = 'path_to_save_trt_model' # 请替换为你想要保存TRT模型的路径
converter.save(saved_model_dir_trt)

# 4. 加载优化后的模型进行推理
root = tf.saved_model.load(saved_model_dir_trt)
concrete_func = root.signatures['serving_default']

# 读取输入数据
test_dir = '/path_to_your_test_data' # 请替换为你的测试数据目录
test_data_files = glob(os.path.join(test_dir, '*.png')) # 假设你的测试数据是PNG图像
test_data = [load_images(file) for file in test_data_files]

# 对每一个输入数据进行推理，并保存结果
results = []
for x in test_data:
    # 增加一个维度以匹配模型的输入要求
    x = np.expand_dims(x, axis=0)
    y = concrete_func(x)
    results.append(y)

# 在这里，results列表中包含了所有输入数据的推理结果


In [ ]:
import tensorflow as tf
from tensorflow.python.compiler.tensorrt import trt_convert as trt
from glob import glob
import numpy as np
import os
from model import *

saved_model_dir = "./model/saved_model"
test_dir = '/simple/low' # 请替换为你的测试数据目录
saved_model_dir_trt = './trt_model' # 请替换为你想要保存TRT模型的路径

# 定义一个函数来读取图像，这个函数需要根据你的实际情况来实现
def load_images(file_name):
    # 这里需要你自己实现如何从file_name读取图像并返回numpy数组
    pass

# 定义一个函数，这个函数将接受所有的输入并返回最后的输出
@tf.function(input_signature=[
    tf.TensorSpec(shape=[None, None, None, 3], dtype=tf.float32),
    tf.TensorSpec(shape=[None, None, None, 3], dtype=tf.float32),
    tf.TensorSpec(shape=[None, None, None, 1], dtype=tf.float32),
    tf.TensorSpec(shape=[None, None, None, 1], dtype=tf.float32),
])
def model_func(input_decom, input_low_i, input_low_i_ratio, input_low_r):
    [R_decom, I_decom] = DecomNet(input_decom)
    output_i, A = Illumination_adjust_curve_net_ratio(input_low_i, input_low_i_ratio)
    output_r = Restoration_net(input_low_r, input_low_i, False)
    return output_i, output_r

# 转换模型
concrete_func = model_func.get_concrete_function()

# 将模型转换为TensorRT模型
params = trt.DEFAULT_TRT_CONVERSION_PARAMS
params = params._replace(max_workspace_size_bytes=(1<<32)) # 修改workspace大小
params = params._replace(precision_mode="FP16") # 设置推理精度
params = params._replace(maximum_cached_engines=100) # 设置最大缓存引擎数量
converter = trt.TrtGraphConverterV2(
    input_saved_model_dir=saved_model_dir,
    conversion_params=params,
    functions=[concrete_func]
)
converter.convert()

# 保存TensorRT模型
converter.save(saved_model_dir_trt)

# 加载优化后的模型进行推理
root = tf.saved_model.load(saved_model_dir_trt)
concrete_func = root.signatures['serving_default']

# 读取输入数据
test_data_files = glob(os.path.join(test_dir, '*.png')) # 假设你的测试数据是PNG图像
test_data = [load_images(file) for file in test_data_files]

# 对每一个输入数据进行推理，并保存结果
results = []
for x in test_data:
    # 增加一个维度以匹配模型的输入要求
    x = np.expand_dims(x, axis=0)
    y = concrete_func(x)
    results.append(y)

# 在这里，results列表中包含了所有输入数据的推理结果
results

In [5]:
import numpy as np
model = tf.saved_model.load("./model/saved_model_trt")

# Define the inference function
infer = model.signatures["serving_default"]

# Assuming your model takes an image as input, prepare a dummy input
# Note: You'll need to adjust the shape and dtype to match your model's expected input.
dummy_input = np.random.randn(1, 400, 600, 3).astype(np.float32)
ratio_tensor = tf.constant(10.0)

# Run inference
output = infer(input_decom=tf.constant(dummy_input),ratio=ratio_tensor)

print(output)

2023-08-06 07:31:04.567680: W tensorflow/compiler/tf2tensorrt/kernels/trt_engine_op.cc:936] TF-TRT Warning: Engine retrieval for input shapes: [[1,400,600,3]] failed. Running native segment for PartitionedCall/TRTEngineOp_000_000


{'fusion4': <tf.Tensor: shape=(1, 1, 400, 600, 3), dtype=float32, numpy=
array([[[[[0.0434414 , 0.03047429, 0.02290096],
          [0.06501768, 0.05392208, 0.03611331],
          [0.07348678, 0.04627388, 0.04862572],
          ...,
          [0.03410219, 0.01493151, 0.01354985],
          [0.03751391, 0.01650799, 0.01170312],
          [0.03895836, 0.01907868, 0.01542505]],

         [[0.04563245, 0.03064032, 0.03568611],
          [0.06169915, 0.04334304, 0.03325764],
          [0.05495221, 0.04613142, 0.04367299],
          ...,
          [0.04187234, 0.01825982, 0.02385909],
          [0.04399507, 0.01707458, 0.01911527],
          [0.02620247, 0.01565142, 0.01592946]],

         [[0.04496914, 0.03003759, 0.02886588],
          [0.05497001, 0.04105491, 0.0275457 ],
          [0.04499306, 0.03976496, 0.03753095],
          ...,
          [0.05955804, 0.02189353, 0.03297251],
          [0.07304651, 0.02460212, 0.03552938],
          [0.03476525, 0.01825411, 0.0195179 ]],

         ...

In [19]:
import time
from utils import save_images
from PIL import Image
import numpy as np
import tensorflow as tf
from glob import glob
import os

def load_images(file):
    im = Image.open(file)
    im = im.resize((600, 400), Image.ANTIALIAS)
    img = np.array(im, dtype="float32") / 255.0
    img_max = np.max(img)
    img_min = np.min(img)
    img_norm = np.float32((img - img_min) / np.maximum((img_max - img_min), 0.001))
    return img_norm

def load_data(test_dir):
    eval_low_data = []
    eval_img_name = []
    eval_low_data_name = glob(os.path.join(test_dir, 'low/*.png'))
    eval_low_data_name.sort()
    for idx in range(len(eval_low_data_name)):
        [_, name] = os.path.split(eval_low_data_name[idx])
        suffix = name[name.find('.') + 1:]
        name = name[:name.find('.')]
        eval_img_name.append(name)
        eval_low_im = load_images(eval_low_data_name[idx])
        eval_low_data.append(eval_low_im)
    return eval_low_data, eval_img_name
test_dir = "./simple"
eval_low_data, eval_img_name = load_data(test_dir) 
for idx in range(len(eval_low_data)):
    name = eval_img_name[idx]
    input_low = eval_low_data[idx]
    input_low_eval = np.expand_dims(input_low, axis=0)
    ratio = float(10.0)
    t1 = time.time()
    output = infer(input_decom=tf.constant(input_low_eval),ratio = tf.constant(10.0))
    t2 = time.time()
    print(f"\033[94minfer time:{t2-t1:.3f}s\033[0m")
    # print(f"{output['fusion4']}")
    save_images(os.path.join("sample_results", '%s_KinD_plus.png' % name), output['fusion4'])

/tmp/ipykernel_2393772/3980427902.py:11: DeprecationWarning: ANTIALIAS is deprecated and will be removed in Pillow 10 (2023-07-01). Use LANCZOS or Resampling.LANCZOS instead.
  im = im.resize((600, 400), Image.ANTIALIAS)


infer time:0.034s
infer time:0.124s
infer time:0.043s
